- How many unique users are there in the dataset?
- Identify the user(s) with the highest number of reviews. What is(are) the average/median review scores for these user(s)?

In [1]:
# setup

import os, sys, findspark
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")
findspark.init()
import pyspark
number_cores = 4
memory_gb = 8
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [2]:
import json

In [47]:
#yelp_small_raw = sc.textFile( './yelp_small.json' )
yelp_full_raw = sc.textFile('./yelp_academic_dataset_user.json')

#yelp_data = yelp_small_raw.map( lambda entry: json.loads(entry) )
yelp_data = yelp_full_raw.map( lambda entry: json.loads(entry) )

def split_friends( entry ):
    entry['friends'] = entry['friends'].split(', ')
    return entry

yelp_data = yelp_data.map( split_friends )
yelp_data.take(1)

[{'user_id': 'ntlvfPzc8eglqvk92iDIAw',
  'name': 'Rafael',
  'review_count': 553,
  'yelping_since': '2007-07-06 03:27:11',
  'useful': 628,
  'funny': 225,
  'cool': 227,
  'elite': '',
  'friends': ['oeMvJh94PiGQnx_6GlndPQ',
   'wm1z1PaJKvHgSDRKfwhfDg',
   'IkRib6Xs91PPW7pon7VVig',
   'A8Aq8f0-XvLBcyMk2GJdJQ',
   'eEZM1kogR7eL4GOBZyPvBA',
   'e1o1LN7ez5ckCpQeAab4iw',
   '_HrJVzFaRFUhPva8cwBjpQ',
   'pZeGZGzX-ROT_D5lam5uNg',
   '0S6EI51ej5J7dgYz3-O0lA',
   'woDt8raW-AorxQM_tIE2eA',
   'hWUnSE5gKXNe7bDc8uAG9A',
   'c_3LDSO2RHwZ94_Q6j_O7w',
   '-uv1wDiaplY6eXXS0VwQiA',
   'QFjqxXn3acDC7hckFGUKMg',
   'ErOqapICmHPTN8YobZIcfQ',
   'mJLRvqLOKhqEdkgt9iEaCQ',
   'VKX7jlScJSA-ja5hYRw12Q',
   'ijIC9w5PRcj3dWVlanjZeg',
   'CIZGlEw-Bp0rmkP8M6yQ9Q',
   'OC6fT5WZ8EU7tEVJ3bzPBQ',
   'UZSDGTDpycDzrlfUlyw2dQ',
   'deL6e_z9xqZTIODKqnvRXQ',
   '5mG2ENw2PylIWElqHSMGqg',
   'Uh5Kug2fvDd51RYmsNZkGg',
   '4dI4uoShugD9z84fYupelQ',
   'EQpFHqGT9Tk6YSwORTtwpg',
   'o4EGL2-ICGmRJzJ3GxB-vw',
   's8gK7sdVzJcYKcP

In [48]:
## number of users
yelp_data.map( lambda entry: entry['user_id'] ).distinct().count()


1968703

In [49]:
user_scores = yelp_data.map( lambda entry: (entry['user_id'], entry['review_count'], entry['average_stars']) )

user_scores.take(5)

[('ntlvfPzc8eglqvk92iDIAw', 553, 3.57),
 ('FOBRPlBHa3WPHFB5qYDlVg', 564, 3.84),
 ('zZUnPeh2hEp0WydbAZEOOg', 60, 3.44),
 ('QaELAmRcDc5TfJEylaaP8g', 206, 3.08),
 ('xvu8G900tezTzbbfqmTKvA', 485, 4.37)]

In [50]:
spark = pyspark.sql.SparkSession(sc)

In [51]:
list(yelp_data.first().keys())

['user_id',
 'name',
 'review_count',
 'yelping_since',
 'useful',
 'funny',
 'cool',
 'elite',
 'friends',
 'fans',
 'average_stars',
 'compliment_hot',
 'compliment_more',
 'compliment_profile',
 'compliment_cute',
 'compliment_list',
 'compliment_note',
 'compliment_plain',
 'compliment_cool',
 'compliment_funny',
 'compliment_writer',
 'compliment_photos']

In [52]:
yelp_data.map(lambda entry: (len(entry['friends']),entry['fans']) ).take(20)

[(45, 14),
 (213, 27),
 (35, 5),
 (173, 6),
 (895, 78),
 (90, 15),
 (54, 0),
 (755, 137),
 (85, 4),
 (344, 31),
 (27, 5),
 (210, 13),
 (1855, 197),
 (8, 0),
 (105, 4),
 (349, 31),
 (296, 11),
 (302, 1),
 (28, 7),
 (262, 26)]

In [54]:
reviews = yelp_data.map(lambda line: (line['review_count'],line ))
reviews.sortByKey(False).map(lambda line: (line[1]['user_id'],line[0],line[1]['average_stars'])).take(1)

[('8k3aO-mPeyhbR5HUucA5aA', 14455, 3.28)]

In [55]:
yelp_data.take(1)

[{'user_id': 'ntlvfPzc8eglqvk92iDIAw',
  'name': 'Rafael',
  'review_count': 553,
  'yelping_since': '2007-07-06 03:27:11',
  'useful': 628,
  'funny': 225,
  'cool': 227,
  'elite': '',
  'friends': ['oeMvJh94PiGQnx_6GlndPQ',
   'wm1z1PaJKvHgSDRKfwhfDg',
   'IkRib6Xs91PPW7pon7VVig',
   'A8Aq8f0-XvLBcyMk2GJdJQ',
   'eEZM1kogR7eL4GOBZyPvBA',
   'e1o1LN7ez5ckCpQeAab4iw',
   '_HrJVzFaRFUhPva8cwBjpQ',
   'pZeGZGzX-ROT_D5lam5uNg',
   '0S6EI51ej5J7dgYz3-O0lA',
   'woDt8raW-AorxQM_tIE2eA',
   'hWUnSE5gKXNe7bDc8uAG9A',
   'c_3LDSO2RHwZ94_Q6j_O7w',
   '-uv1wDiaplY6eXXS0VwQiA',
   'QFjqxXn3acDC7hckFGUKMg',
   'ErOqapICmHPTN8YobZIcfQ',
   'mJLRvqLOKhqEdkgt9iEaCQ',
   'VKX7jlScJSA-ja5hYRw12Q',
   'ijIC9w5PRcj3dWVlanjZeg',
   'CIZGlEw-Bp0rmkP8M6yQ9Q',
   'OC6fT5WZ8EU7tEVJ3bzPBQ',
   'UZSDGTDpycDzrlfUlyw2dQ',
   'deL6e_z9xqZTIODKqnvRXQ',
   '5mG2ENw2PylIWElqHSMGqg',
   'Uh5Kug2fvDd51RYmsNZkGg',
   '4dI4uoShugD9z84fYupelQ',
   'EQpFHqGT9Tk6YSwORTtwpg',
   'o4EGL2-ICGmRJzJ3GxB-vw',
   's8gK7sdVzJcYKcP

In [57]:
ready_to_df_data = yelp_data.map(lambda entry: (entry['user_id'], entry['review_count'], entry['yelping_since'], entry['average_stars'], \
                                            entry['useful'] + \
                                            entry['funny'] + \
                                            entry['cool'] + \
                                            entry['compliment_hot'] + \
                                            entry['compliment_more'] + \
                                            entry['compliment_cute'] + \
                                            entry['compliment_list'] + \
                                            entry['compliment_note'] + \
                                            entry['compliment_plain'] + \
                                            entry['compliment_cool'] + \
                                            entry['compliment_funny'] + \
                                            entry['compliment_writer'] + \
                                            entry['compliment_photos'] ) )

df = spark.createDataFrame( ready_to_df_data , ['id','reviews','joined','ave_stars','interactions'] )
df.show()

+--------------------+-------+-------------------+---------+------------+
|                  id|reviews|             joined|ave_stars|interactions|
+--------------------+-------+-------------------+---------+------------+
|ntlvfPzc8eglqvk92...|    553|2007-07-06 03:27:11|     3.57|        1166|
|FOBRPlBHa3WPHFB5q...|    564|2008-04-28 01:29:25|     3.84|        1771|
|zZUnPeh2hEp0WydbA...|     60|2008-08-28 23:40:05|     3.44|         444|
|QaELAmRcDc5TfJEyl...|    206|2008-09-20 00:08:14|     3.08|         517|
|xvu8G900tezTzbbfq...|    485|2008-08-09 00:30:27|     4.37|        2358|
|z5_82komKV3mI4ASG...|    186|2007-02-27 07:09:40|     2.88|        1215|
|ttumcu6hWshk_EJVW...|     12|2010-05-12 16:30:08|      4.0|          45|
|f4_MRNHvN-yRn7EA8...|    822|2011-01-17 00:18:23|     3.63|       12519|
|UYACF30806j2mfbB5...|     14|2007-07-24 23:55:21|     3.75|         136|
|QG13XBbgHWydzThRB...|    218|2007-06-04 00:14:10|      4.1|        1575|
|f6YuZP6iennHFVlnF...|      5|2008-08-

In [58]:
df.describe().show()

+-------+--------------------+-----------------+-------------------+------------------+-----------------+
|summary|                  id|          reviews|             joined|         ave_stars|     interactions|
+-------+--------------------+-----------------+-------------------+------------------+-----------------+
|  count|             1968703|          1968703|            1968703|           1968703|          1968703|
|   mean|                null|  22.169298771831|               null|3.6480868216282665|93.11573304861119|
| stddev|                null|76.74225931902373|               null|1.1725254942007215|1701.675754092096|
|    min|---1lKK3aKOuomHnw...|                0|2004-10-12 08:40:43|               1.0|                0|
|    max|zzzv9W8ShM4TkXZGM...|            14455|2019-12-13 15:46:07|               5.0|           678694|
+-------+--------------------+-----------------+-------------------+------------------+-----------------+



In [59]:
#df['popularity score'] = df['average stars'] / df['reviews']

df = df.withColumn('popularity', df.reviews / df.ave_stars )
df.describe('popularity').show()

+-------+------------------+
|summary|        popularity|
+-------+------------------+
|  count|           1968703|
|   mean| 6.073808353647311|
| stddev|20.579686644611897|
|    min|               0.0|
|    max| 4407.012195121952|
+-------+------------------+



In [60]:
df.orderBy('popularity', ascending = False ).show()

+--------------------+-------+-------------------+---------+------------+------------------+
|                  id|reviews|             joined|ave_stars|interactions|        popularity|
+--------------------+-------+-------------------+---------+------------+------------------+
|8k3aO-mPeyhbR5HUu...|  14455|2007-12-08 14:56:45|     3.28|      261673| 4407.012195121952|
|hWDybu_KvYLSdEFzG...|  12487|2009-03-08 21:47:44|     3.64|      286758|3430.4945054945056|
|RtGqdDBvvBCjcu5dU...|  12772|2010-10-17 06:35:06|     3.87|       18078|3300.2583979328165|
|Hi10sGSZNxQH3NLyW...|  11112|2009-05-26 11:33:58|      3.8|      451875|2924.2105263157896|
|P5bUL3Engv-2z6kKo...|   9875|2006-05-31 21:27:42|      3.8|       63364| 2598.684210526316|
|8RcEwGrFIgkt9WQ35...|   7745|2009-11-06 22:53:16|     3.49|       15891|2219.1977077363895|
|nmdkHL2JKFx55T3nq...|   7626|2011-11-29 15:50:53|     3.71|      123834| 2055.525606469003|
|Xwnf20FKuikiHcSpc...|   6762|2011-06-10 03:52:07|     3.32|       184

# Open-Ended Section

- Identify several (two or more) interesting user attributes and think of and carry out one interesting analysis. Interesting is relative, and you should use Markdown cell to describe why/how these are interesting!
- Identify the top-ten most influential users from the Yelp dataset. Justify the attribute(s) that you pick to measure the level of influence!  

### Frequently Active Users
* The three new fields that I am adding to the dataframe calculated from the existing data are: 
    * `monthsDiff`: the total time (in months) that the user has had a registered account.
    * `interactivity`: the average amount of interactions per month that a user accrues
    * `activity`: the average reviews per month that a user posts


In [77]:

from pyspark.sql.functions import *

df = df.withColumn('monthsDiff', months_between(current_date(), col('joined')))
df = df.withColumn('interactivity', df.interactions / df.monthsDiff )
df = df.withColumn('activity', df.reviews / df.monthsDiff )

df['activity','interactions','popularity','monthsDiff','interactivity'].describe().show()


+-------+-------------------+-----------------+------------------+-----------------+------------------+
|summary|           activity|     interactions|        popularity|       monthsDiff|     interactivity|
+-------+-------------------+-----------------+------------------+-----------------+------------------+
|  count|            1968703|          1968703|           1968703|          1968703|           1968703|
|   mean|0.18532363858962572|93.11573304861119| 6.073808353647311|101.7583855787262|0.6872480320248533|
| stddev| 0.5443923926279562|1701.675754092096|20.579686644611897|33.28375920426219|12.057380047310707|
|    min|                0.0|                0|               0.0|      34.59170886|               0.0|
|    max|  88.40282552551677|           678694| 4407.012195121952|     216.63349649|  5696.51172259726|
+-------+-------------------+-----------------+------------------+-----------------+------------------+



## Top users by category:
* Interactivity (interactions per month)
* activity (posts per month)
* popularity (reviews / ave stars)

In [80]:
df.orderBy('interactivity', ascending = False )['activity','ave_stars','interactivity','popularity'].show()

+------------------+---------+------------------+------------------+
|          activity|ave_stars|     interactivity|        popularity|
+------------------+---------+------------------+------------------+
|12.573228230175447|     4.17|  5696.51172259726| 359.2326139088729|
| 7.626074803487403|     4.14|3903.1729304483833|291.54589371980677|
| 6.292502730434553|     3.58| 3088.174232984465| 270.1117318435754|
| 68.94241040890368|      3.8|2803.5773671277316|2924.2105263157896|
|12.024096385542169|     4.27| 2304.385542168675|467.44730679156913|
|13.699523953889086|     3.72|2252.9008083775066| 653.2258064516129|
| 19.17854386786478|     3.64|2218.2825638529844| 787.6373626373626|
|20.273878382930544|     3.53|2132.0529838364596| 575.0708215297451|
| 10.30044707760467|     4.21| 2125.469979136571|152.49406175771972|
|  19.1445588403082|     4.45|2121.9164325053785| 621.3483146067415|
|17.502613690023875|     3.09|1939.0729092296183| 946.2783171521036|
|2.6972566041244055|     4.21|1773

In [85]:
df.orderBy('activity', ascending = False )['activity','ave_stars','interactivity','popularity'].show()

+------------------+---------+------------------+------------------+
|          activity|ave_stars|     interactivity|        popularity|
+------------------+---------+------------------+------------------+
| 88.40282552551677|     3.87|125.12889757675323|3300.2583979328165|
| 80.86527576610385|     3.28|1463.8712767584707| 4407.012195121952|
| 76.25823840026285|     3.64| 1751.234077615326|3430.4945054945056|
| 68.94241040890368|      3.8|2803.5773671277316|2924.2105263157896|
|  58.1802210828016|     3.71| 944.7534090699781| 2055.525606469003|
|50.126035446699625|      3.8| 321.6390997513595| 2598.684210526316|
|49.708632787940985|     3.49|101.99094688614205|2219.1977077363895|
| 49.46436367887847|     3.32|134.94803033835404|2036.7469879518073|
|44.447785855121694|     4.01|174.25611483785724| 852.8678304239402|
|42.597311832962376|     3.33| 371.9767067675029|1991.8918918918919|
|40.306235414990034|     3.59|  81.4044910682827|1034.8189415041784|
| 38.01846850726704|     3.77|324.

In [81]:
df.orderBy('popularity', ascending = False)['activity','ave_stars','interactivity','popularity'].show()

+------------------+---------+------------------+------------------+
|          activity|ave_stars|     interactivity|        popularity|
+------------------+---------+------------------+------------------+
| 80.86527576610385|     3.28|1463.8712767584707| 4407.012195121952|
| 76.25823840026285|     3.64| 1751.234077615326|3430.4945054945056|
| 88.40282552551677|     3.87|125.12889757675323|3300.2583979328165|
| 68.94241040890368|      3.8|2803.5773671277316|2924.2105263157896|
|50.126035446699625|      3.8| 321.6390997513595| 2598.684210526316|
|49.708632787940985|     3.49|101.99094688614205|2219.1977077363895|
|  58.1802210828016|     3.71| 944.7534090699781| 2055.525606469003|
| 49.46436367887847|     3.32|134.94803033835404|2036.7469879518073|
|42.597311832962376|     3.33| 371.9767067675029|1991.8918918918919|
|29.395526060814902|     3.93| 581.1014484523929|1399.4910941475825|
|28.976847396989935|     3.46|63.398967747553016|1382.6589595375722|
| 34.00279210146395|     3.39| 888

### Statistical Correlations

* Columns:
    * id
    * reviews             
    * joined         
    * ave_stars      
    * interactions        
    * popularity        
    * monthsDiff      
    * interactivity

## Does the frequency of user activity affect popularity?
- If the most popular users are also the most frequently active users, then we should see a strong correlation between interactivity and popularity 
- However, as we see, the correlation is quite strong. This indicates that the most popular users are the ones that post the most frequently.

In [82]:
df.stat.corr("popularity","activity")

0.9567655162180551

## Does the most popular users get the most interactions on their posts?
- One would expect that by the most popular users yield the most interaction from their followers, be they various compliments, upvotes, etc.
- However, the statistics indicates that this is only a moderately-strong positive correlation.

In [64]:
df.stat.corr("popularity","interactivity")

0.4482313448590867

## Does more activity yield more interactivity?
- We can see that this is only a moderately-strong positive correlation between the activity and interactivity between the user's posts.

In [84]:
df.stat.corr('activity','interactivity')

0.45131264456811226


## Does popularity correlate to the average stars on the user's posts?
- As we saw before, there is virtually no correlation between the user's popularity and positive responses

In [66]:
df.stat.corr("popularity","ave_stars")

0.0004773504646774203

## Are the most popular users also the oldest? 
* This is not strongly supported by the data. As we can see, there is only a weak positive correlation.

In [67]:
df.stat.corr("popularity","monthsDiff")

0.24854270065644915

## Does interactivity correspond to 
* No, there is no statistically significant indication that 'better' posts get more interactions.

In [86]:
df.stat.corr("ave_stars","interactivity")

0.008622195410370851